In [1]:
import numpy as np
import math
import cvxpy as cp
from itertools import combinations
import time
import copy
import pandas as pd
import copy

In [2]:
np.random.seed(1)

In [3]:
def collection_distribution(n,collection):
    #collection = list(combinations(range(n),size))
    collection_distribution = np.zeros((n,len(collection)))

    for i in range(len(collection)):
        distribution_S = np.random.exponential(1, len(collection[i]))
        assortment_dis = np.array([k/np.sum(distribution_S) for k in distribution_S])
        for j in range(len(collection[i])):
            collection_distribution[collection[i][j]][i]= assortment_dis[j]
            
    return collection_distribution

In [4]:
n = 7
instance_size = 100
#collection_size = [2,3,5,10,15,20]
collection_size = [2,3,5,10,15]
#collection_size = [2,3,5]

all_size2_ass = [list(x) for x in list(combinations(range(n),2))]
all_size3_ass = [list(x) for x in list(combinations(range(n),3))]
#all_size4_ass = [list(x) for x in list(combinations(range(n),4))]
#all_possible_assortment = all_size2_ass + all_size3_ass + all_size4_ass
all_possible_assortment = all_size2_ass + all_size3_ass 

all_collection = [[] for _ in range(len(collection_size))]

# construct 1000 samples of random assortment size collection of first collection size
for j in range(instance_size):
    collection = []
    while(len(collection)<collection_size[0]):
        S = all_possible_assortment[np.random.randint(len(all_possible_assortment))]
        if S not in collection:
            collection.append(S)
    all_collection[0].append(collection)


for i in range(1,len(collection_size)):
    all_collection[i] = copy.deepcopy(all_collection[i-1])
    for j in range(instance_size):
        while(len(all_collection[i][j])<collection_size[i]):
            S = all_possible_assortment[np.random.randint(len(all_possible_assortment))]
            if S not in all_collection[i][j]:
                all_collection[i][j].append(S)

all_instance = []
for i in range(len(collection_size)):
    collection_ins = []   
    for j in range(instance_size):
        collection_ins.append(collection_distribution(n,all_collection[i][j]))
        ''' v = np.random.normal(0,1,size=n) #standard normal
        curr_coll_mnl = collection_distribution_mnl(n,all_collection[i][j],v)
        collection_ins.append(gaussian_noise(curr_coll_mnl,std,alpha)) '''
    all_instance.append(collection_ins)


In [5]:
data = all_instance[0][0]

In [6]:
def data_process(data):
    not_row = data[[not np.all(data[i] == 0) for i in range(data.shape[0])], :]  ### only keep the rows where the data points are not all zero meaning that the product is included in the assortments
    
    pre_collection = copy.deepcopy(not_row.T)
    collection = []
    for i in range(len(pre_collection)):
        collection.append(list(np.nonzero(pre_collection[i])[0]))

    binary_pre_collection = copy.deepcopy(not_row.T)
    binary_pre_collection[binary_pre_collection >0] =1
    binary_pre_collection[binary_pre_collection ==0] =-5000000
    binary_pre_collection = binary_pre_collection.astype(int)
    binary_collection = binary_pre_collection.tolist()
    
    return not_row,collection,binary_collection 

In [7]:
data_process(data)

(array([[0.81484605, 0.72923242],
        [0.11280148, 0.        ],
        [0.07235248, 0.09655199],
        [0.        , 0.17421559]]),
 [[0, 1, 2], [0, 2, 3]],
 [[1, 1, 1, -5000000], [1, -5000000, 1, 1]])

In [8]:
result = data_process(data)
print(result[0].shape)
print(len(result[1]))
print(len(result[2]))

(4, 2)
2
2


In [8]:
from cvxpy import log_sum_exp

def mnl_mle(data,curr_collection,collection):
    #print(data)
    n = data.shape[0]
    v= cp.Variable(n)
    ''' test_v = list(range(n)) 
    test_v = [x+1 for x in test_v]
    print('testing',test_v) '''
    #print(len(curr_collection))
    #collection = [np.array([1,1,1,0]),np.array([1,0,1,1]),np.array([1,0,0,1])]
    obj = 0
    for i in range(len(curr_collection)): 
        for j in curr_collection[i]:
            #print(cp.log_sum_exp(np.array(collection[i]).T@test_v))
            #print(cp.log(np.array(collection[i])@ cp.exp(test_v)))
            #print(cp.log(cp.sum(cp.multiply(np.array(collection[i]),cp.exp(test_v)),keepdims=True)))
            #obj = obj + data[j][i]* (v[j] - cp.log(cp.sum(cp.multiply(np.array(collection[i]),cp.exp(v)),keepdims=True)))
            #print('testing',cp.multiply(collection[i],test_v))
            #print('check collection',collection[i])
            obj = obj + data[j][i]*v[j]
        obj = obj - cp.log_sum_exp(cp.multiply(collection[i],v))
            #obj = obj + data[j][i]* (v[j] - cp.log_sum_exp(cp.multiply(collection[i],v)))
         
    prob = cp.Problem(cp.Maximize(obj))
    prob.solve()
    #print(v.value)
    return v.value

In [9]:
#data

In [10]:
#test_data = np.array([[1/3,1/3,1/3,0],[1/3,0,1/3,1/3]]).T

''' test_data = np.array([[1/2,1/2,0,0,0],[0,0,1/3,1/3,1/3]]).T
test_data.shape '''

' test_data = np.array([[1/2,1/2,0,0,0],[0,0,1/3,1/3,1/3]]).T\ntest_data.shape '

In [11]:
#mnl_mle(test_data, [[0,1],[2,3,4]], [[1,1,0,0,0],[0,0,1,1,1]])


In [12]:
#pro_data, collection, b_collection = data_process(test_data)
''' print(pro_data)
for i in range(len(collection)): 
    for j in collection[i]:
        print(pro_data[j][i]) '''
        #obj = obj + data[j][i]* (v[j] - cp.log_sum_exp(v))

#mnl_mle(pro_data, collection, b_collection)


' print(pro_data)\nfor i in range(len(collection)): \n    for j in collection[i]:\n        print(pro_data[j][i]) '

In [13]:
def mnl_ditribution(v,collection):
    collection_distribution = np.zeros((len(v),len(collection)))
    for i in range(len(collection)):
        curr_total_v = 0
        for j in collection[i]:
            curr_total_v = curr_total_v + math.exp(v[j])
        for k in range(len(collection[i])):
            collection_distribution[collection[i][k]][i] = math.exp(v[collection[i][k]])/curr_total_v 
    return collection_distribution  

In [14]:
def compute_loss(mnl_data,sample_data):
    loss = 0
    for i in range(sample_data.shape[0]):
        for j in range(sample_data.shape[1]):
            #loss = loss + np.abs(sample_data[i][j]-mnl_data[i][j])
            loss = loss + sample_data[i][j] * np.abs(sample_data[i][j]-mnl_data[i][j])
    return loss

In [15]:
print('starting testing the limit of MNL')
start_time = time.time()
mnl_runtime = []
mnl_total_loss = []  #of different collection size 
mnl_mean_loss = []
mnl_loss_se = []
mnl_runtime_se = []
for i in range(len(all_instance)):
    loss = []
    runtime = [] # of the same collection size
    for j in range(len(all_instance[i])):
        start_run_time = time.time()
        curr_instance,collection,binary_collection = data_process(all_instance[i][j])
        v = mnl_mle(curr_instance,collection,binary_collection)
        if len(v)==0:
            continue
        estimated_mnl_data = mnl_ditribution(v,collection)
        curr_loss = compute_loss(estimated_mnl_data,curr_instance)
        used_time = time.time() - start_run_time
        loss.append(curr_loss)
        runtime.append(used_time)
        
    test = pd.DataFrame({'loss':loss,'runtime':runtime})
    test.to_csv('mnl/collection_'+str(collection_size[i])+'.csv')
    runtime = np.array(runtime)
    mnl_runtime.append(np.mean(runtime))
    mnl_runtime_se.append(np.std(runtime)/np.sqrt(len(runtime)))
    loss = np.array(loss)
    mnl_mean_loss.append(np.mean(loss))
    mnl_total_loss.append(np.sum(loss))
    mnl_loss_se.append(np.std(loss)/np.sqrt(len(loss)))
    
print('the total running time',time.time()-start_time)

starting testing the limit of MNL
the total running time 24.63560700416565


In [16]:
mnl = pd.DataFrame({'collection_size':collection_size,'mnl_mean_oss':mnl_mean_loss,'mnl_loss_se':mnl_loss_se,'mnl_mean_runtime':mnl_runtime,'mnl_runtime_se':mnl_runtime_se})
mnl.to_csv('mnl_mle_summary.csv')

In [17]:
mnl_mean_loss

[0.025551576765766824,
 0.13534370594345396,
 0.4990971863587697,
 1.6802910061161078,
 2.8867146205778997]

In [18]:
mnl_loss_se

[0.008150273132167957,
 0.019855244328794885,
 0.028390855764469397,
 0.03711771081375096,
 0.05771037135164159]